# **COURSEWORK (22COP504)** Programming for Data Science
# By student no: F216727 [Start Date:03/01/23]

## 1.2 Store cleaned data (Preprocessing)

### Functions

### COA111 cleaning and preparations

In [1]:
# import all the neccessarily libraries
import pandas as pd
import numpy as np
import sqlite3

# create a function and read coa111module .csv file
def read_coa111():

    file = "22COA111ModuleRegister.csv" 
    path = "./cop504cwdata/" 
    dfcoa111 = pd.read_csv(path+file,index_col = 0) # reading the raw coa111 file 
    
    dfcleancoa111 = dfcoa111  
    clm = dfcleancoa111.columns.to_list() # change the column headers to a list 
    information = [] # strip to remove spaces
    
    for item in clm:
        item = item.strip() 
        information.append(item) 

    dfcleancoa111.isnull().sum() 
    dfcleancoa111.columns = np.arange(len(dfcleancoa111.columns)) # set a new column header by numbering orderly
    dfcleancoa111 = dfcleancoa111.replace({'Ex': np.nan}) 
    dfcleancoa111 = dfcleancoa111.dropna(how='all', axis=0) # drop the missing values  by rows 
    dfcleancoa111 = dfcleancoa111.dropna(how='all', axis=1) # drop the missing values by columns
    dfcleancoa111 = dfcleancoa111.replace({'GPS' : True, 'X' : False}) 
    dfcleancoa111 = dfcleancoa111.transpose() # switch all column headers with the index row
    dfcleancoa111.index.names = ["session_no"] 
    
    display(dfcleancoa111) 
    
    # create another dataframe to store the details of each sessions
    dfCOA111details = pd.DataFrame(information, columns = ["details"])
    dfCOA111details = dfCOA111details["details"].str.split("\n", expand = True) 
    dfCOA111details.columns = ["week", "day_date", "time", "type", "room"] 
    dfCOA111details[['semester', 'week']] = dfCOA111details["week"].str.split(".", expand = True)
    dfCOA111details.index.names = ["session_no"] 

    dfcoa111sessions = dfCOA111details 

    display(dfcoa111sessions)

    return (dfcleancoa111,dfcoa111sessions) 

### COA122 cleaning and preparations

In [2]:
def read_coa122():

    file = "22COA122ModuleRegister.csv" 
    path = "./cop504cwdata/" 
    dfcoa122 = pd.read_csv(path+file,index_col = 0)  

    dfcleancoa122 = dfcoa122 
    clm = dfcleancoa122.columns.to_list() 
    information = [] # strip to remove spaces
    
    for item in clm:
        item = item.strip()
        information.append(item)
    
    dfcleancoa122.isnull().sum() 
    dfcleancoa122.columns = np.arange(len(dfcleancoa122.columns)) 
    dfcleancoa122 = dfcleancoa122.replace({'Ex' : np.nan}) 
    dfcleancoa122 = dfcleancoa122.dropna(how='all', axis=0) 
    dfcleancoa122 = dfcleancoa122.dropna(how='all', axis=1) 
    dfcleancoa122 = dfcleancoa122.replace({'GPS' : True, 'X' : False}) 
    dfcleancoa122 = dfcleancoa122.transpose() 
    dfcleancoa122.index.names = ["session_no"] 

    display(dfcleancoa122) 
    
    # create a new df to store the details of each sessions 
    dfCOA122details = pd.DataFrame(information, columns = ["details"])  
    dfCOA122details = dfCOA122details["details"].str.split("\n", expand = True)
    dfCOA122details.columns = ["week", "day", "time", "type", "room", "drop_in"]
    dfCOA122details[['semester', 'week']] = dfCOA122details["week"].str.split(".", expand = True)
    dfCOA122details.drop(columns=["drop_in"], inplace = True)
    dfCOA122details.index.names = ["session_no"]
    dfcoa122sessions = dfCOA122details 

    display(dfcoa122sessions) 
    
    return (dfcleancoa122,dfcoa122sessions)

### Database operations

In [3]:
# create a function for writing to the database
def write_to_db(dfData, TableName):
    conn = sqlite3.connect('CWDatabase.db') 
    dfData.to_sql(TableName,if_exists = 'replace',
                  index = True,
                  con = conn)
    conn.close() # 

# read dfcleancoa111 & dfcoa111sessions from database
def get_dfcleancoa111_from_db():
    conn = sqlite3.connect('CWDatabase.db')
    dfcleancoa111 = pd.read_sql("select * from stucoa111", 
                            con = conn,
                            index_col = ['session_no'])
    return dfcleancoa111

def get_dfcoa111sessions_from_db():
    conn = sqlite3.connect('CWDatabase.db')
    dfcoa111sessions = pd.read_sql("select * from coa111sessions", 
                            con = conn,
                            index_col = ['session_no'])
    return dfcoa111sessions

# read dfcleancoa122 & dfcoa122sessions from database
def get_dfcleancoa122_from_db():
    conn = sqlite3.connect('CWDatabase.db')
    dfcleancoa122 = pd.read_sql("select * from stucoa122",
                            con = conn,
                            index_col = ['session_no'])
    return dfcleancoa122

def get_dfcoa122sessions_from_db():
    conn = sqlite3.connect('CWDatabase.db')
    dfcoa122sessions = pd.read_sql("select * from coa122sessions",
                            con = conn,
                            index_col = ['session_no'])
    return dfcoa122sessions

### Main function (reading and writing to DB)

In [4]:
# create the main function for reading and writing to the database
def main():
    # reading dfcleancoa111 & dfcoa111sessions
    dfcleancoa111,dfcoa111sessions = read_coa111()
    dfcleancoa111.index.names = ['session_no'] 

    # reading dfcleancoa122 & dfcoa122sessions
    dfcleancoa122,dfcoa122sessions = read_coa122()
    dfcleancoa122.index.names = ['session_no']

    # writing two dataframes of coa111 into the DB
    write_to_db(dfcleancoa111,"stucoa111") 
    write_to_db(dfcoa111sessions, "coa111sessions") 

    # writing two dataframes of coa122 into the DB
    write_to_db(dfcleancoa122, "stucoa122") 
    write_to_db(dfcoa122sessions, "coa122sessions") 

### Testing (all dataframes)

#### Testing coa111 

In [5]:
dfcleancoa111,dfcoa111sessions = read_coa111() 
# the first table shows the details of each student. 
# the second table displays the details of each sessions. 

sid_x,0,1,2,3,4,5,6,7,8,9,...,233,234,235,236,237,238,239,240,241,242
session_no,,,,,,,,,,,,,,,,,,,,,
0,True,True,True,NaN,True,True,True,True,True,True,...,True,True,True,True,True,False,True,True,True,False
1,True,True,True,NaN,False,True,False,True,False,True,...,NaN,True,True,True,False,False,False,True,True,False
2,False,True,True,True,False,True,False,True,False,True,...,False,True,False,True,True,True,True,True,True,False
4,True,True,False,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,False
5,True,True,False,False,False,True,False,True,True,True,...,True,True,False,True,False,False,True,False,True,False
7,True,True,True,True,True,True,True,True,True,False,...,True,True,False,True,True,True,True,True,True,False
8,True,True,False,False,False,True,False,True,False,False,...,True,True,True,True,True,True,True,True,True,NaN
10,False,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,NaN
11,False,True,False,False,False,True,True,True,False,True,...,False,True,True,True,True,True,True,True,True,NaN


,week,day_date,time,type,room,semester
session_no,,,,,,
0,W1,Monday (03-10-2022),09:00 - 10:00,Lecture,U020,S1
1,W1,Wednesday (05-10-2022),10:00 - 11:00,Lecture,SMB014,S1
2,W2,Wednesday (12-10-2022),10:00 - 11:00,Lecture,SMB014,S1
3,W2,Wednesday (12-10-2022),12:00 - 13:00,Tutorial,None,S1
4,W2,Thursday (13-10-2022),14:00 - 15:00,Lecture,CC011,S1
5,W3,Wednesday (19-10-2022),10:00 - 11:00,Lecture,SMB014,S1
6,W3,Wednesday (19-10-2022),12:00 - 13:00,Tutorial,None,S1
7,W3,Thursday (20-10-2022),14:00 - 15:00,Lecture,CC011,S1
8,W4,Wednesday (26-10-2022),10:00 - 11:00,Lecture,SMB014,S1


#### Testing coa122

In [6]:
dfcleancoa122,dfcoa122sessions = read_coa122()

sid,0,1,2,3,4,5,6,7,8,9,...,233,234,235,236,237,238,239,240,241,242
session_no,,,,,,,,,,,,,,,,,,,,,
0,True,True,True,NaN,True,True,True,True,True,True,...,True,True,True,True,True,False,True,True,True,False
1,True,True,True,NaN,True,NaN,NaN,True,True,NaN,...,NaN,NaN,NaN,NaN,NaN,False,True,NaN,True,NaN
2,NaN,NaN,NaN,NaN,NaN,True,True,NaN,NaN,True,...,True,True,True,False,False,NaN,NaN,True,NaN,False
3,True,True,True,NaN,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,False
4,False,True,True,False,False,NaN,NaN,True,False,NaN,...,NaN,NaN,NaN,NaN,NaN,True,True,NaN,True,NaN
5,NaN,NaN,NaN,False,NaN,True,False,NaN,NaN,True,...,True,True,True,True,True,NaN,NaN,True,NaN,NaN
6,False,True,True,False,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,False
7,False,True,True,NaN,False,NaN,NaN,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,True,False,NaN,True,NaN
8,NaN,NaN,NaN,True,NaN,True,False,NaN,NaN,True,...,True,True,False,True,True,NaN,NaN,True,NaN,NaN


,week,day,time,type,room,semester
session_no,,,,,,
0,W1,Monday (03-10-2022),14:00 - 16:00,Lecture,CC012,S1
1,W1,Thursday (06-10-2022),09:00 - 11:00,Computer Lab,N001...,S1
2,W1,Thursday (06-10-2022),11:00 - 13:00,Computer Lab,N001...,S1
3,W2,Monday (10-10-2022),14:00 - 16:00,Lecture,CC012,S1
4,W2,Thursday (13-10-2022),09:00 - 11:00,Computer Lab,N001...,S1
5,W2,Thursday (13-10-2022),11:00 - 13:00,Computer Lab,N001...,S1
6,W3,Monday (17-10-2022),14:00 - 16:00,Lecture,CC012,S1
7,W3,Thursday (20-10-2022),09:00 - 11:00,Computer Lab,N001...,S1
8,W3,Thursday (20-10-2022),11:00 - 13:00,Computer Lab,N001...,S1
